<a href="https://colab.research.google.com/github/axefx/DS-Unit-2-Applied-Modeling/blob/master/module2-wrangle-ml-datasets/Axel_Corro_LS_DS13_232_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Read New York City property sales data
import pandas as pd
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

Your code starts here:

In [5]:
# Change column names: replace spaces with underscores
df.columns = df.columns.str.replace(" ","_")
df.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'SALE_DATE'],
      dtype='object')

In [0]:
# Get Pandas Profiling Report
# from pandas_profiling import ProfileReport
# ProfileReport(df)

In [7]:
# Keep just the subset of data for the Tribeca neighborhood
# Check how many rows you have now. (Should go down from > 20k rows to 146)
tribeca = df[df['NEIGHBORHOOD']=='TRIBECA'].reset_index()
tribeca.head()

,index,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,220,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,222,1013,NaN,R4,"181 HUDSON STREET, 6D",6D,10013.0,1.0,0.0,1.0,"7,878",1840.0,1909.0,2,R4,"$ 2,800,000",01/03/2019
1,763,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,9022,NaN,R4,"10 LITTLE WEST STREET, 19A",19A,10004.0,1.0,0.0,1.0,0,1759.0,0.0,2,R4,"$ 2,650,000",01/07/2019
2,996,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,212,1404,NaN,R4,"11 BEACH STREET, 1D",1D,10013.0,1.0,0.0,1.0,"11,213",2651.0,1900.0,2,R4,$ - 0,01/08/2019
3,1276,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,1441,NaN,R4,"30 LITTLE WEST STREET, 7G",7G,10004.0,1.0,0.0,1.0,0,1190.0,2005.0,2,R4,"$ 1,005,000",01/09/2019
4,1542,1,TRIBECA,02 TWO FAMILY DWELLINGS,1,132,26,NaN,S2,"75 WARREN STREET, XX",NaN,10007.0,2.0,1.0,3.0,"1,819",10117.0,1905.0,1,S2,"$ 12,950,000",01/10/2019


In [8]:
# Q. What's the date range of these property sales in Tribeca?
tribeca['SALE_DATE'].describe()

count            146
unique            66
top       02/12/2019
freq              17
Name: SALE_DATE, dtype: object

In [9]:
# The Pandas Profiling Report showed that SALE_PRICE was read as strings
# Convert it to integers
tribeca['SALE_DATE'] = pd.to_datetime(tribeca['SALE_DATE'],infer_datetime_format=True)
tribeca['SALE_DATE'].describe()[['first','last']]

first    2019-01-03 00:00:00
last     2019-04-30 00:00:00
Name: SALE_DATE, dtype: object

In [10]:
# Q. What is the maximum SALE_PRICE in this dataset?
tribeca['SALE_PRICE'].head()

0     $   2,800,000
1     $   2,650,000
2           $   - 0
3     $   1,005,000
4    $   12,950,000
Name: SALE_PRICE, dtype: object

In [0]:
# Look at the row with the max SALE_PRICE
def clean_saleprice(content):
  content = content.replace(" ","").replace("$","").replace(",","")
  content = pd.to_numeric(content)
  return content

tribeca['SALE_PRICE'] = tribeca['SALE_PRICE'].apply(clean_saleprice)

In [12]:
tribeca.sort_values(by='SALE_PRICE').tail(1)

,index,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
33,6499,1,TRIBECA,08 RENTALS - ELEVATOR APARTMENTS,2,224,1,NaN,D8,34 DESBROSSES STREET,NaN,10013.0,283.0,3.0,286.0,"36,858",305542.0,2007.0,2,D8,260000000,2019-02-01


In [13]:
# Get value counts of TOTAL_UNITS
# Q. How many property sales were for multiple units?
tribeca['TOTAL_UNITS'].value_counts()

1.0      131
0.0       11
5.0        1
286.0      1
8.0        1
3.0        1
Name: TOTAL_UNITS, dtype: int64

In [0]:
# Keep only the single units
tribeca_singles = tribeca[tribeca['TOTAL_UNITS'] < 2]

In [15]:
# Q. Now what is the max sales price? How many square feet does it have?
tribeca_singles.sort_values(by='SALE_PRICE').tail(1)['GROSS_SQUARE_FEET']

75    8346.0
Name: GROSS_SQUARE_FEET, dtype: float64

In [16]:
# Q. How often did $0 sales occur in this subset of the data?

# There's a glossary here: 
# https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page

# It says:
# A $0 sale indicates that there was a transfer of ownership without a 
# cash consideration. There can be a number of reasons for a $0 sale including 
# transfers of ownership from parents to children. 

no_cash_transfers = tribeca[tribeca['SALE_PRICE'] == 0]
len(no_cash_transfers)

19

In [17]:
# Look at property sales for > 5,000 square feet
# Q. What is the highest square footage you see?
tribeca[tribeca['GROSS_SQUARE_FEET'] > 5000].sort_values(by='GROSS_SQUARE_FEET').tail(1)

,index,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
33,6499,1,TRIBECA,08 RENTALS - ELEVATOR APARTMENTS,2,224,1,NaN,D8,34 DESBROSSES STREET,NaN,10013.0,283.0,3.0,286.0,"36,858",305542.0,2007.0,2,D8,260000000,2019-02-01


In [18]:
# What are the building class categories?
# How frequently does each occur?
tribeca['BUILDING_CLASS_CATEGORY'] = tribeca['BUILDING_CLASS_CATEGORY'].str.replace(" ","_")
tribeca['BUILDING_CLASS_CATEGORY'].value_counts()

13_CONDOS_-_ELEVATOR_APARTMENTS               121
10_COOPS_-_ELEVATOR_APARTMENTS                  9
15_CONDOS_-_2-10_UNIT_RESIDENTIAL               8
17_CONDO_COOPS                                  2
08_RENTALS_-_ELEVATOR_APARTMENTS                2
02_TWO_FAMILY_DWELLINGS                         1
07_RENTALS_-_WALKUP_APARTMENTS                  1
16_CONDOS_-_2-10_UNIT_WITH_COMMERCIAL_UNIT      1
46_CONDO_STORE_BUILDINGS                        1
Name: BUILDING_CLASS_CATEGORY, dtype: int64

In [24]:
# Keep subset of rows:
# Sale price more than $0, 
# Building class category = Condos - Elevator Apartments

# Check how many rows you have now. (Should be 106 rows.)
tribeca_condos = tribeca[(tribeca['SALE_PRICE'] != 0) & (tribeca['BUILDING_CLASS_CATEGORY'] == '13_CONDOS_-_ELEVATOR_APARTMENTS')]
tribeca_condos

,index,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,220,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,222,1013,NaN,R4,"181 HUDSON STREET, 6D",6D,10013.0,1.0,0.0,1.0,"7,878",1840.0,1909.0,2,R4,2800000,2019-01-03
1,763,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,16,9022,NaN,R4,"10 LITTLE WEST STREET, 19A",19A,10004.0,1.0,0.0,1.0,0,1759.0,0.0,2,R4,2650000,2019-01-07
3,1276,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,16,1441,NaN,R4,"30 LITTLE WEST STREET, 7G",7G,10004.0,1.0,0.0,1.0,0,1190.0,2005.0,2,R4,1005000,2019-01-09
5,1543,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,16,3624,NaN,R4,"2 RIVER TERRACE, 4C",4C,0.0,1.0,0.0,1.0,0,1088.0,2006.0,2,R4,1678000,2019-01-10
6,1855,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,16,2610,NaN,R4,"380 RECTOR PLACE, 10A",10A,10280.0,1.0,0.0,1.0,0,873.0,0.0,2,R4,1380000,2019-01-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,22221,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,222,1206,NaN,R4,"443 GREENWICH STREET, 1F",1F,10013.0,1.0,0.0,1.0,"35,127",2429.0,1905.0,2,R4,5761259,2019-04-24
142,22732,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,16,1806,NaN,R4,"70 LITTLE WEST STREET, 11K",11K,10004.0,1.0,0.0,1.0,0,1601.0,2006.0,2,R4,2600000,2019-04-29
143,22733,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,16,8071,NaN,R4,"2 SOUTH END AVENUE, 5G",5G,10280.0,1.0,0.0,1.0,0,634.0,1990.0,2,R4,605000,2019-04-29
144,22897,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,16,8132,NaN,R4,"2 SOUTH END AVENUE, 8B",8B,10280.0,1.0,0.0,1.0,0,939.0,1990.0,2,R4,960000,2019-04-30


In [25]:
# Make a Plotly Express scatter plot of GROSS_SQUARE_FEET vs SALE_PRICE
import plotly.express as px
px.scatter(tribeca_condos, x='GROSS_SQUARE_FEET',y='SALE_PRICE')

In [26]:
# Add an OLS (Ordinary Least Squares) trendline,
# to see how the outliers influence the "line of best fit"
px.scatter(tribeca_condos, x='GROSS_SQUARE_FEET',y='SALE_PRICE',trendline='ols')

In [27]:
# Look at sales for more than $35 million

# All are at 70 Vestry Street
# All but one have the same SALE_PRICE & SALE_DATE
# Was the SALE_PRICE for each? Or in total?
# Is this dirty data?
tribeca_condos[tribeca_condos['SALE_PRICE'] > 35000000]

,index,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
57,8370,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,223,1105,NaN,R4,"70 VESTRY STREET, 3C",3C,10013.0,1.0,0.0,1.0,0,1670.0,2016.0,2,R4,36681561,2019-02-12
58,8371,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,223,1123,NaN,R4,"70 VESTRY STREET, 6C",6C,10013.0,1.0,0.0,1.0,0,1906.0,2016.0,2,R4,36681561,2019-02-12
59,8372,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,223,1124,NaN,R4,"70 VESTRY STREET, 6D",6D,10013.0,1.0,0.0,1.0,0,2536.0,2016.0,2,R4,36681561,2019-02-12
60,8373,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,223,1125,NaN,R4,"70 VESTRY STREET, 6E",6E,10013.0,1.0,0.0,1.0,0,2965.0,2016.0,2,R4,36681561,2019-02-12
61,8374,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,223,1126,NaN,R4,"70 VESTRY STREET, 6F",6F,10013.0,1.0,0.0,1.0,0,2445.0,2016.0,2,R4,36681561,2019-02-12
62,8375,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,223,1127,NaN,R4,"70 VESTRY STREET, 7A",7A,10013.0,1.0,0.0,1.0,0,2844.0,2016.0,2,R4,36681561,2019-02-12
63,8376,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,223,1128,NaN,R4,"70 VESTRY STREET, 7B",7B,10013.0,1.0,0.0,1.0,0,3242.0,2016.0,2,R4,36681561,2019-02-12
64,8377,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,223,1129,NaN,R4,"70 VESTRY STREET, 7C",7C,10013.0,1.0,0.0,1.0,0,1906.0,2016.0,2,R4,36681561,2019-02-12
65,8378,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,223,1130,NaN,R4,"70 VESTRY STREET, 7D",7D,10013.0,1.0,0.0,1.0,0,2536.0,2016.0,2,R4,36681561,2019-02-12
66,8379,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,223,1131,NaN,R4,"70 VESTRY STREET, 7E",7E,10013.0,1.0,0.0,1.0,0,2965.0,2016.0,2,R4,36681561,2019-02-12


In [30]:
# Make a judgment call:
# Keep rows where sale price was < $35 million

# Check how many rows you have now. (Should be down to 90 rows.)
tribeca_condos = tribeca_condos[tribeca_condos['SALE_PRICE'] < 35000000]
tribeca_condos

,index,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,220,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,222,1013,NaN,R4,"181 HUDSON STREET, 6D",6D,10013.0,1.0,0.0,1.0,"7,878",1840.0,1909.0,2,R4,2800000,2019-01-03
1,763,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,16,9022,NaN,R4,"10 LITTLE WEST STREET, 19A",19A,10004.0,1.0,0.0,1.0,0,1759.0,0.0,2,R4,2650000,2019-01-07
3,1276,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,16,1441,NaN,R4,"30 LITTLE WEST STREET, 7G",7G,10004.0,1.0,0.0,1.0,0,1190.0,2005.0,2,R4,1005000,2019-01-09
5,1543,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,16,3624,NaN,R4,"2 RIVER TERRACE, 4C",4C,0.0,1.0,0.0,1.0,0,1088.0,2006.0,2,R4,1678000,2019-01-10
6,1855,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,16,2610,NaN,R4,"380 RECTOR PLACE, 10A",10A,10280.0,1.0,0.0,1.0,0,873.0,0.0,2,R4,1380000,2019-01-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,22221,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,222,1206,NaN,R4,"443 GREENWICH STREET, 1F",1F,10013.0,1.0,0.0,1.0,"35,127",2429.0,1905.0,2,R4,5761259,2019-04-24
142,22732,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,16,1806,NaN,R4,"70 LITTLE WEST STREET, 11K",11K,10004.0,1.0,0.0,1.0,0,1601.0,2006.0,2,R4,2600000,2019-04-29
143,22733,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,16,8071,NaN,R4,"2 SOUTH END AVENUE, 5G",5G,10280.0,1.0,0.0,1.0,0,634.0,1990.0,2,R4,605000,2019-04-29
144,22897,1,TRIBECA,13_CONDOS_-_ELEVATOR_APARTMENTS,2,16,8132,NaN,R4,"2 SOUTH END AVENUE, 8B",8B,10280.0,1.0,0.0,1.0,0,939.0,1990.0,2,R4,960000,2019-04-30


In [31]:
# Now that you've removed outliers,
# Look again at a scatter plot with OLS (Ordinary Least Squares) trendline
px.scatter(tribeca_condos, x='GROSS_SQUARE_FEET',y='SALE_PRICE',trendline='ols')

In [0]:
# Select these columns, then write to a csv file named tribeca.csv. Don't include the index.
tribeca_condos[['GROSS_SQUARE_FEET','SALE_PRICE']].to_csv('tribeca.csv')